## 有隐状态的循环神经网络

In [32]:
import torch
from d2l import torch as d2l

X, W_xh = torch.normal(0,1,(3,1)),torch.normal(0,1,(1,4))
X, W_xh

(tensor([[ 1.8266],
         [-0.1622],
         [ 0.1962]]),
 tensor([[-0.7801, -1.0291, -0.6613,  1.1083]]))

In [33]:
H, W_hh = torch.normal(0, 1, (3,4)), torch.normal(0,1,(4,4))
H, W_hh

(tensor([[-0.0271, -0.5243, -0.9061,  0.7240],
         [ 0.1195, -0.3175,  1.9111, -1.1962],
         [ 0.3077, -1.4439, -1.0198, -1.2417]]),
 tensor([[ 0.7498,  0.3606, -1.6225,  0.2844],
         [-0.8399,  0.8504,  0.9477, -0.8297],
         [-1.6136,  1.5098,  0.1830,  1.4897],
         [ 0.0806, -1.3204,  0.5920,  2.4908]]))

In [34]:
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.5154, -4.6592, -1.3979,  2.9054],
        [-2.6974,  4.4048, -0.7460, -0.0149],
        [ 2.8361, -1.2190, -2.9191, -3.1089]])

In [35]:
torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh),dim=0))

tensor([[ 0.5154, -4.6592, -1.3979,  2.9054],
        [-2.6974,  4.4048, -0.7460, -0.0149],
        [ 2.8361, -1.2190, -2.9191, -3.1089]])

公式:
$H_t=\phi(X_tW_{xh}+H_{t-1}W_hh+b_h)$

In [36]:
torch.cat((X, H), dim=1).shape

torch.Size([3, 5])

In [37]:
torch.cat((W_xh, W_hh),dim=0).shape

torch.Size([5, 4])

equation:
$\frac{1}{n}\mathop\Sigma_{t=1}^n-logP(x_t|x_{t-1},...,x_1)$

perplexity:
$exp(\frac{1}{n}\mathop\Sigma_{t=1}^n-logP(x_t|x_{t-1},...,x_1))$

In [38]:
%matplotlib
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

Using matplotlib backend: TkAgg


In [39]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [40]:
F.one_hot(torch.tensor([0,3]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [41]:
X = torch.arange(10).reshape((2,5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

In [42]:
F.one_hot(X.T, 28)[-1]

tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [43]:
print(X.T)

tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8],
        [4, 9]])


In [44]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [45]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [46]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

状态:
$H=\phi(XW_{xh}+b_h)$
$O=HW_{hq}+b_q$

# 从零开始实现的循环神经网络模型

In [47]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [48]:
num_hiddens = 1024
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 1024]))

In [49]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [50]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

'time traveller fqldvjpwtk'

In [51]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [52]:
#@save
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [53]:
#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('time machine'))

In [56]:
num_epochs, lr = 1000, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

困惑度 1.0, 92339.0 词元/秒 cuda:0
time traveller for so it will be convenient to speak of himwas e
time machine by h g wellsithe time traveller for so it will be


In [55]:
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(),
          use_random_iter=True)

困惑度 1.5, 87397.0 词元/秒 cuda:0
time traveller held in his hand was a glitteringmetallic framewo
time machine by h g wellsithe time traveller proceeded anyreal
